In [1]:
if(!dir.exists("Functions/")){
    setwd("../")
    if(!dir.exists("Functions")){
        setwd("M:/lecospec/lecospec/")
    }
}
source("Functions/lecospectR.R", echo = FALSE)

Loading required package: tidyverse

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: compiler

Loading required package: raster

Loading required package: sp


Attaching package: ‘raster’


The following object is masked from ‘package:dplyr’:

    select


Loading required package: hsdar

Loading required package: rgdal

Please note that rgdal will be retired during 2023,
plan transition to sf/stars/terra functions using GDAL and PROJ
at your earliest convenience.
See https://r-spatial.org/r/2

In [2]:
log_model_results <- function(model_id, confusion_matrix, distribition, custom = NULL, logpath = "./gs.log"){
    # append performance data to the logs for later comparison
    sink(file = logpath, append = TRUE)
    print("-------------------------------------------------------")
    print("---------------------- Model Data ---------------------")
    
    print(paste0("Model Type: PLS-LDA (plsgenomics)"))
    print(paste0("Data Index: ",custom))
    print(paste0("Model UUID: ", model_id))
    print("---------------------- Confusion Matrix ---------------------")
    print(confusion_matrix)
    print("---------------------- Class Distribution ---------------------")
    print(distribition)
    print("-------------------------------------------------------")
    sink(NULL)
}

In [3]:
add_model_to_manifest <- function(
    model_id, 
    outlier = "", 
    preprocessing="",
    source="", 
    weight = "",
    n = "",
    oob_error = "",
    accuracy = "",
    r2 = "",
    chi2prob = "",
    logpath="./gs_manifest.csv"){
    if(!file.exists(logpath)){
        header <- "source,outliers,preprocessing,weight,n,oob,accuracy,r2,rpd,model_id"
        write(header, file = logpath)
    }

    line <- paste(
        source,
        outlier,
        preprocessing,
        weight,
        n,
        oob_error,
        accuracy,
        r2,
        chi2prob,
        sep=","
    )
    line <- paste0(line, ",", model_id)

    write(line, file=logpath, append = TRUE)
}

In [4]:
train_pls_lda <- function(
    train_df, 
    train_labels,
    test_df, 
    test_labels,
    ncomp = 32,
    outlier_fn = NULL,
    preprocess_fn = NULL,
    weight_fn = targets_to_weights,
    model_id = uuid::UUIDgenerate(),
    ignore_cols = NULL,
    save_path = "./mle/models",
    seed = NULL,
    log_string = ""
){

    if(!is.null(seed)){
        set.seed(seed)
    }

    x_train <- train_df %>% as.data.frame()
    x_test <- test_df %>% as.data.frame()
    if(is.function(outlier_fn)){
        x_train <- outlier_fn(x_train)
    }
    if(is.function(preprocess_fn)){
        x_train <- preprocess_fn(x_train)
        x_test <- preprocess_fn(x_test)
    }

    sample_perm <- permute::shuffle(length(train_labels))
    sample_index <- create_stratified_sample(
        train_labels,
        permutation = sample_perm,
        samples_per_pft = 300
    )
    x_train <- x_train[sample_perm,][sample_index,]
    train_labels <- train_labels[sample_perm][sample_index]

    #if(("Forb" %in% levels(train_labels)) && !("Forb"  %in% levels(test_labels))){
        levels(test_labels) <- c(levels(test_labels), "Forb")
    #    }
    
    model_s3_obj <- list(
        x_train = x_train,
        y_train = train_labels,
        ncomp = ncomp,
        nruncv = 0
    )

    class(model_s3_obj) <- "pls_lda"

    save(
        model_s3_obj,
        file = file.path(save_path, paste0(model_id, ".rda"))
    )

    # create predictions (ranger)
    model_predictions <- predict(
            model_s3_obj, 
            x_test
        ) 

    print(model_predictions)


    # generate the confusion matrix

    confusion_matrix <- caret::confusionMatrix(
        model_predictions %>% as.factor(),
        test_labels,
        mode = "everything"
    )

    log_model_results(
            model_id = model_id,
            confusion_matrix = confusion_matrix,
            custom = log_string,
            distribition = model_predictions %>% as.factor() %>% table(),
            logpath = "./gs_rf_n.log")

    return(
        list(
                model = model_s3_obj,
                confusion = confusion_matrix %>% as.list()
                )
    )

    

}

In [5]:
apply_model.pls_lda <- function(x, model,  ...){
    
    if("x" %in% colnames(x) && "y" %in% colnames(x)){
        print("Spatial ifnormation detected, ignoring....")
        target_df <- subset(x, select=-c(x,y))
        prediction_df <- predict.pls_lda(model, target_df) %>% as.data.frame()
        prediction_df$x <- x$x
        prediction_df$y <- x$y

    } else {
        prediction_df <- predict.pls_lda(model, x) %>% as.data.frame()
    }

    return(prediction_df)
}

In [6]:
write_pls_lda_model <- function(object, save_path, uuid = NULL){
    model_id <- uuid
    if(is.null(model_id)){
        model_id <- uuid::UUIDgenerate()
    }

    save(x_train, x_train_path)
}

In [7]:
predict.pls_lda <- function(pls_lda, x, ...){

    predctions <- plsgenomics::pls.lda(
        pls_lda$x_train,
        pls_lda$y_train %>% as.numeric() + 1,
        as.data.frame(x),
        pls_lda$ncomp,
        pls_lda$nruncv
    )$predclass

    predictions_df <- as.data.frame(as.numeric(predctions) - 1)
    colnames(predictions_df) <- c("z")
    predictions_df$x <- rep(0,nrow(predictions_df))
    predictions_df$y <- rep(0,nrow(predictions_df))


    return(convert_fg1_int( predictions_df)$z %>% as.factor())
    
}

In [8]:
filter_target_subset <- function(df){

    df_cols <- colnames(df) %>% as.character()
    target_cols <- c(
        "Vogelmann",
        "TVI",
        "CRI4",
        "Datt5",
        "DWSI4",
        "GDVI",
        "MCARI",
        "MTVI",
        "NPCI",
        "PARS",
        "X417.593_5nm",
        "X787.593_5nm",
        "X892.593_5nm"
    )

    used_cols <- intersect(df_cols, target_cols)
    print(paste0("Filtering Data frame to the following columns: ", used_cols))
    return(df[,used_cols])
}

In [9]:
read_pls_lda_model <- function(pls_lda_obj){
    x_train

}

In [10]:
train_model <- function(
    train_df, 
    train_labels,
    test_df, 
    test_labels,
    ntree = 50,
    max_depth = NULL,
    mtry = NULL,
    outlier_fn = NULL,
    preprocess_fn = NULL,
    weight_fn = targets_to_weights,
    model_id = uuid::UUIDgenerate(),
    ignore_cols = NULL,
    seed = NULL,
    log_string = ""
){
    if(!is.null(seed)){
        set.seed(seed)
    }

    x_train <- train_df %>% as.data.frame()
    x_test <- test_df %>% as.data.frame()
    if(is.function(outlier_fn)){
        x_train <- outlier_fn(x_train)
    }
    if(is.function(preprocess_fn)){
        x_train <- preprocess_fn(x_train)
        x_test <- preprocess_fn(x_test)
    }

    sample_perm <- permute::shuffle(length(train_labels))
    sample_index <- create_stratified_sample(
        train_labels,
        permutation = sample_perm,
        samples_per_pft = 300
    )
    x_train <- x_train[sample_perm,][sample_index,]
    train_labels <- train_labels[sample_perm][sample_index] %>% as.factor()

    model <- ranger::ranger(
            num.trees = ntree,
            mtry = mtry,
            max.depth = max_depth,
            #case.weights = weight_fn(train_labels),
            classification = TRUE,
            x=x_train,
            y=train_labels
        )

    if(("Forb" %in% levels(train_labels)) && !("Forb"  %in% levels(test_labels))){
            levels(test_labels) <- c(levels(test_labels), "Forb")
            }

    # create predictions (ranger)
        model_predictions <- predict(
            model, 
            x_test
        )$prediction %>% as.factor()

        # generate the confusion matrix
        print(model_predictions %>% levels())
        print(test_labels %>% levels())
print(model_predictions %>% to_fg0() %>% levels())
print(test_labels %>% to_fg0() %>% levels())


        confusion_matrix <- caret::confusionMatrix(
            model_predictions %>% to_fg0(), 
            test_labels %>% as.factor() %>% to_fg0(),
            mode = "everything"
        )

        # generate an id to uniquely identify the model
        #model_id <- uuid::UUIDgenerate()

        # append performance data to the logs for later comparison
        log_model_results(
            model_id = model_id,
            confusion_matrix = confusion_matrix,
            custom = log_string,
            distribition = model_predictions %>% as.factor() %>% table(),
            logpath = "./gs_pls_lda.log")

        # track what levels are associated with the UUID

        # save the model using the model UUID
        save(model, file = paste0("mle/models/gs/", model_id, ".rda"))
        
        return(
            list(
                model = model,
                confusion = confusion_matrix %>% as.list()
                )
        )
}

In [11]:
base_paths <- c(
    "img_raw_raw.csv",
    "corrected_and_img.csv",
    "grd_raw_raw.csv",
    "bison_gulch_stratified.csv",
    "grd_raw_corrected.csv",
    "img_indices_only.csv",# include veg indices
    "grd_indices_only.csv"
)

In [12]:
calculate_posterior_weights <- function(validation_path ="figures/merged_validation_s.csv" ){

    validation_df <- read.csv(validation_path, header = TRUE)
    #print(head(validation_df))

    total_observations <- sum(validation_df$validation_counts)
    #print(total_observations)
    weights <- (1/ validation_df$validation_prop)
    #print(validation_df$validation_prop)

    total_by_fg1 <- aggregate(
        x = validation_df$validation_counts,
        by = list(validation_df$key),
        FUN = sum
    )

    fg1_weight_list <- list()

    for( row_idx in seq(nrow(total_by_fg1))){
        name <- total_by_fg1$Group.1[[row_idx]]
        value <- total_by_fg1$x[[row_idx]]
        fg1_weight_list[name] <- value
    }
    
    return(fg1_weight_list)
}

get_posterior_weights_from_targets <- function(target_factor, posterior_weight = calculate_posterior_weights()){
    unbiased_weights <- targets_to_weights(target_factor)

    target_name_char <- target_factor %>% as.character()

    output_weights <- seq_along(target_factor)

    for(i in seq_along(target_factor)){
        if(posterior_weight[[target_name_char[[i]]]] > 0){
            fg1_weight <- 1 / posterior_weight[[target_name_char[[i]]]]
        } else {
            fg1_weight <- 0
        }
        output_weights[[i]] <- unbiased_weights[[i]] * fg1_weight
    }

    return(output_weights)
}

## Hypotheses

Peter
* Increasing number of trees will improve the accuracy/validation mismatch
* bias in 

Ken
* decreasing the number of the trees will decrease the impact of the posterior weighting on chi-squared statistic

In [13]:
outlier_functions <- list(
    #clip = load_model("./mle/clip_transform.rda"),
    no_treatment = function(x, ignore_cols = NULL){return(x)}# no transform
)

outlier_treatments <- c(
    "no_treatment"#,
    #"clip"
)

preprocess_functions <- list(
    no_treatment = function(x, ignore_cols = NULL){return(x)},# no transform
    min_max = columnwise_min_max_scale,
    robust = columnwise_robust_scale,
    standard = standardize_df
)

weight_functions <- list(
    posterior = get_posterior_weights_from_targets,
    balanced = targets_to_weights,
    no_treatment = function(x){return(NULL)}# No weights
)
weight_treatments <- c(
    #"no_treatment"#,
    "balanced"#,
    #"posterior"
)

preprocessing_treatments <- c(
    "no_treatment"#,
    #"min_max",
    #"standard",
    #"robust"
)

In [14]:
num_components <- c(
    1,
    2,
    #4,6,
    8,#
    10,
    12,#14,
    16,#18,20,24,
    32,#50,
    64,#100,
    128,
    256,
    512#,1024
)

depth_control <- c(
    #10,20, 
    
    40
)

In [15]:
to_fg0 <- function(target_factor){
    return(
        change_aggregation(
            target_factor,
            0,
            rjson::fromJSON(file="./assets/pft_adj_list.json")
        ) %>% 
        as.factor()
    )
}

In [16]:
test_data <- subset(read.csv("Data/gs/x_test/img_raw_raw.csv"), select = -c(X))
test_labels <- read.csv("Data/gs/y_test/img_raw_raw.csv")$x %>% as.factor()
#train_labels <- read.csv("Data/gs/y_train/img_raw_raw.csv")$x %>% as.factor()
manifest_path <- "./gs_manifest_reduced_val.csv"

In [17]:
for(filepath in base_paths){
    train_data <- subset(read.csv(paste0("Data/gs/x_train/", filepath)), select = -c(X))
    labels <- read.csv(paste0("Data/gs/y_train/", filepath))$x %>% as.factor()



    for(o_treatment in outlier_treatments){
        for(p_treatment in preprocessing_treatments){
            for(w_treatment in weight_treatments){
                for(n in num_components){
#                    for(d in depth_control){

                model_id <- uuid::UUIDgenerate()
                save_path <- paste0("mle/experiments/gs/", model_id, "/")
                if(!dir.exists(save_path)){
                    dir.create(save_path)
                }

                rf_model_results <- train_model(
                    #filter_df_bands(train_data), 
                    train_data,
                    labels, 
                    test_data,
                    test_labels,
                    ntree = n,
                    max_depth = NULL,#d,
                    outlier_fn = outlier_functions[[o_treatment]],
                    preprocess_fn = preprocess_functions[[p_treatment]],
                    weight_fn = weight_functions[[w_treatment]],
                    model_id = model_id,
                    seed=61718,
                    log_string = paste(n, filepath, o_treatment, p_treatment, w_treatment)
                )
                #print(rf_model_results)

                rf_model <- rf_model_results$model
                acc <- as.list(rf_model_results$confusion$overall)$Accuracy
                #print(acc)

                if(acc > 0.6){


                results <- validate_model(
                    rf_model, 
                    save_path, 
                    outlier_processing = outlier_functions[[o_treatment]],
                    transform_type = preprocess_functions[[p_treatment]],
                    pft_aggregation = 0
                )

                aggregated_results <- aggregate_results(save_path)

                # calculate validation statistics
                #chi2 <- calculate_chi_squared_probability(aggregated_results)
                r2 <- calculate_validation_r2(aggregated_results)
                rpd <- calculate_rpd(aggregated_results)

                add_model_to_manifest(
                    model_id = model_id,
                    outlier = o_treatment,
                    preprocessing = p_treatment,
                    source = filepath,
                    weight = w_treatment,
                    n = n,
                    oob_error = rf_model$prediction.error,
                    accuracy = acc,
                    r2 = r2,
                    chi2prob = rpd,
                    logpath=manifest_path
                )

                plot_by_pft(
                    aggregated_results,
                    save_path = paste0(save_path, "aggregate.html"),
                    open = FALSE,
                    image_path = NULL,
                    aggregation=0
                )
            #
                write_validation_table(
                    aggregated_results,
                    save_path = paste0(save_path, "table.html"),
                    open = FALSE
                )
                } else {
                    add_model_to_manifest(
                        model_id = model_id,
                        outlier = o_treatment,
                        preprocessing = p_treatment,
                        source = filepath,
                        weight = w_treatment,
                        n = n,
                        oob_error = rf_model$prediction.error,
                        accuracy = acc,
                        r2 = "Skipped",
                        chi2prob = "Skipped",
                        logpath=manifest_path
                    )

                }
                   # }
                }
            }
        }
    }
}

[1] "ShrubEvergreen" "ShrubDecid"     "TreeConifer"    "TreeConifer"   
[5] "Abiotic"       
[1] "Abiotic"        "Graminoid"      "Lichen"         "Moss"          
[5] "ShrubDecid"     "ShrubEvergreen" "TreeBroadleaf"  "TreeConifer"   
[1] "Abiotic"        "Graminoid"      "Lichen"         "Moss"          
[5] "ShrubDecid"     "ShrubEvergreen" "TreeBroadleaf"  "TreeConifer"   
[1] "Abiotic"          "BroadleafDecid"   "ConiferEvergreen" "Graminoid"       
[5] "Lichen"           "Moss"            
[1] "Abiotic"          "BroadleafDecid"   "ConiferEvergreen" "Graminoid"       
[5] "Lichen"           "Moss"            
[1] "preprocessing raster at Data/Ground_Validation/Imagery/BisonGulchQuads.envi"
[1] "Converted to Data frame?"
[1] TRUE
  [1] "x"        "y"        "X397.593" "X399.444" "X401.296" "X403.148"
  [7] "X405"     "X406.851" "X408.703" "X410.555" "X412.407" "X414.258"
 [13] "X416.11"  "X417.962" "X419.814" "X421.666" "X423.517" "X425.369"
 [19] "X427.221" "X429.073" "X430.924

Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.

`.cols` has been renamed and is deprecated, please use `.vars`
Warning message:
“`funs()` was deprecated in dplyr 0.8.0.
ℹ Please use a list of either functions or lambdas:

# Simple named list: list(mean = mean, median = median)

# Auto named with `tibble::lst()`: tibble::lst(mean, median)

# Using lambdas list(~ mean(., trim = .2), ~ median(., na.rm = TRUE))
ℹ The deprecated feature was likely used in the useful package.
  Please report the issue at <https://github.com/jaredlander/useful/issues>.”


[1] "Handling Outliers with User supplied function"
[1] "Transforming Data with user supplied functions"


`.cols` has been renamed and is deprecated, please use `.vars`


Reading layer `Bisoon_Quadrats_georeferenced' from data source 
  `/home/krbundy/GitHub/lecospec/Data/Vectors/Bisoon_Quadrats_georeferenced.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 9 features and 3 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -148.9524 ymin: 63.80698 xmax: -148.9508 ymax: 63.80701
Geodetic CRS:  WGS 84


...

Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.

`.cols` has been renamed and is deprecated, please use `.vars`


[1] "Handling Outliers with User supplied function"
[1] "Transforming Data with user supplied functions"


`.cols` has been renamed and is deprecated, please use `.vars`


Reading layer `Bisoon_Quadrats_georeferenced' from data source 
  `/home/krbundy/GitHub/lecospec/Data/Vectors/Bisoon_Quadrats_georeferenced.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 9 features and 3 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -148.9524 ymin: 63.80698 xmax: -148.9508 ymax: 63.80701
Geodetic CRS:  WGS 84


Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image


In [ ]:
print(change_aggregation)

function (prediction_vec, aggregation_level, aggregation_key) 
{
    prediction_char <- prediction_vec %>% as.character()
    updated_predictions <- vector("character", length = length(prediction_vec))
    for (i in seq_along(prediction_vec)) {
        aggregation_idx <- 5 - aggregation_level
        prediction <- prediction_char[[i]]
        updated_predictions[[i]] <- aggregation_key[[prediction]][[aggregation_idx]]
    }
    return(updated_predictions)
}
<bytecode: 0x5604340561e8>


In [ ]:
rjson::fromJSON(file = "assets/pft_adj_list.json")

$`dead salix`
[1] "Dead Salix" "LeafLitter" "Litter"     "Abiotic"    "Abiotic"   

$`Dead Salix`
[1] "Dead Salix" "LeafLitter" "Litter"     "Abiotic"    "Abiotic"   

$LeafLitter
[1] "NA"         "LeafLitter" "Litter"     "Abiotic"    "Abiotic"   

$Litter
[1] "NA"      "NA"      "Litter"  "Abiotic" "Abiotic"

$Abiotic
[1] "NA"      "NA"      "NA"      "Abiotic" "Abiotic"

$`spruce bark`
[1] "Pices (bark)" "Wood"         "Litter"       "Abiotic"      "Abiotic"     

$`Pices (bark)`
[1] "Pices (bark)" "Wood"         "Litter"       "Abiotic"      "Abiotic"     

$Wood
[1] "NA"      "Wood"    "Litter"  "Abiotic" "Abiotic"

$`bare rock`
[1] "Bare Rock" "Rock"      "Mineral"   "Abiotic"   "Abiotic"  

$`Bare Rock`
[1] "Bare Rock" "Rock"      "Mineral"   "Abiotic"   "Abiotic"  

$Rock
[1] "NA"      "Rock"    "Mineral" "Abiotic" "Abiotic"

$Mineral
[1] "NA"      "NA"      "Mineral" "Abiotic" "Abiotic"

$gravel
[1] "Bare Soil" "Rock"      "Mineral"   "Abiotic"   "Abiotic"  

$`Bare Soil`
[1] "Bare Soil" "Rock"      "Mineral"   "Abiotic"   "Abiotic"  

$quartz
[1] "Quartz"  "Rock"    "Mineral" "Abiotic" "Abiotic"

$Quartz
[1] "Quartz"  "Rock"    "Mineral" "Abiotic" "Abiotic"

$bare_soil
[1] "Bare Soil" "Soil"      "Mineral"   "Abiotic"   "Abiotic"  

$Soil
[1] "NA"      "Soil"    "Mineral" "Abiotic" "Abiotic"

$equarv
[1] "Equisetum arvense" "Equisetum"         "FernAlly"         
[4] "Forb"              "Forb"             

$`Equisetum arvense`
[1] "Equisetum arvense" "Equisetum"         "FernAlly"         
[4] "Forb"              "Forb"             

$Equisetum
[1] "NA"        "Equisetum" "FernAlly"  "Forb"      "Forb"     

$FernAlly
[1] "NA"       "NA"       "FernAlly" "Forb"     "Forb"    

$Forb
[1] "NA"   "NA"   "NA"   "Forb" "Forb"

$equsyl
[1] "Equisetum sylvaticum" "Equisetum"            "FernAlly"            
[4] "Forb"                 "Forb"                

$`Equisetum sylvaticum`
[1] "Equisetum sylvaticum" "Equisetum"            "FernAlly"            
[4] "Forb"                 "Forb"                

$EQAR
[1] "Equisetum arvense" "Equisetum"         "FernAlly"         
[4] "Forb"              "Forb"             

$DOFR
[1] "Arenaria pseudofrigida" "Arenia"                 "ForbFlower"            
[4] "Forb"                   "Forb"                  

$`Arenaria pseudofrigida`
[1] "Arenaria pseudofrigida" "Arenia"                 "ForbFlower"            
[4] "Forb"                   "Forb"                  

$Arenia
[1] "NA"         "Arenia"     "ForbFlower" "Forb"       "Forb"      

$ForbFlower
[1] "NA"         "NA"         "ForbFlower" "Forb"       "Forb"      

$herlan
[1] "Heracleum lanatum" "Heracleum"         "ForbFlower"       
[4] "Forb"              "Forb"             

$`Heracleum lanatum`
[1] "Heracleum lanatum" "Heracleum"         "ForbFlower"       
[4] "Forb"              "Forb"             

$Heracleum
[1] "NA"         "Heracleum"  "ForbFlower" "Forb"       "Forb"      

$HIERA
[1] "Hieracium sp." "Hieracium"     "ForbFlower"    "Forb"         
[5] "Forb"         

$`Hieracium sp.`
[1] "Hieracium sp." "Hieracium"     "ForbFlower"    "Forb"         
[5] "Forb"         

$Hieracium
[1] "NA"         "Hieracium"  "ForbFlower" "Forb"       "Forb"      

$irisit
[1] "Iris sp."   "Iris"       "ForbFlower" "Forb"       "Forb"      

$`Iris sp.`
[1] "Iris sp."   "Iris"       "ForbFlower" "Forb"       "Forb"      

$Iris
[1] "NA"         "Iris"       "ForbFlower" "Forb"       "Forb"      

$luparc
[1] "Lupinus sp." "Lupinus"     "ForbFlower"  "Forb"        "Forb"       

$`Lupinus sp.`
[1] "Lupinus sp." "Lupinus"     "ForbFlower"  "Forb"        "Forb"       

$Lupinus
[1] "NA"         "Lupinus"    "ForbFlower" "Forb"       "Forb"      

$pedrac
[1] "Pedicularis racemosa" "Pedicularis"          "ForbFlower"          
[4] "Forb"                 "Forb"                

$`Pedicularis racemosa`
[1] "Pedicularis racemosa" "Pedicularis"          "ForbFlower"          
[4] "Forb"                 "Forb"                

$Pe